In [18]:
##환경설정
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random, copy, time

#재현성
SEED = 2025
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print("PyTorch version : ",torch.__version__)
print("Device : ",torch.device("cuda"if torch.cuda.is_available()else"cpu"))

PyTorch version :  2.8.0+cpu
Device :  cpu


In [19]:
##DataSet
N = 1000
X = np.random.randn(N, 2).astype(np.float32)
true_w=np.array([[2.5],[-1.3]],dtype=np.float32)
true_b=np.array([0.7],dtype=np.float32)

y=X@true_w+true_b+0.5*np.random.randn(N,1).astype(np.float32)

X_train,y_train = torch.from_numpy(X[:800]), torch.from_numpy(y[:800])
X_val,y_val = torch.from_numpy(X[800:]), torch.from_numpy(y[800:])

In [20]:
##Define model
class LinearReg(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(2, 1)
    def forward(self,x):
        return self.lin(x)

import torch.nn.functional as F

def mse_loss(pred, target):
        return F.mse_loss(pred, target)
def make_minibatches(X, y, batch_size, generator=None):
        n = X.shape[0]
        perm = torch.randperm(n) if generator is None else torch.randperm(n,generator=generator)
        for i in range(0, n, batch_size):
            j = perm[i:i+batch_size]
            yield X[j], y[j]

In [24]:
##학습루프
##BGD
model = LinearReg()
lr = 1e-2

# 전체 데이터로 한번에 forward/backward/step
pred = model(X_train)
loss = mse_loss(pred, y_train)
loss.backward()

with torch.no_grad():
    for p in model.parameters():
        p -= lr * p.grad
        p.grad.zero_()

val = mse_loss(model(X_val), y_val).item()
print("BGD one-epoch | train:", loss.item(), "val:", val)


BGD one-epoch | train: 9.614413261413574 val: 9.436654090881348


In [16]:
##학습루프
##SGD
model = LinearReg()
lr = 1e-2

perm = torch.randperm(X_train.shape[0])
total = 0.0
for i in range(X_train.shape[0]):
    xi = X_train[perm[i]:perm[i]+1]
    yi = y_train[perm[i]:perm[i]+1]
    pred = model(xi)
    loss = mse_loss(pred, yi)
    loss.backward()
    with torch.no_grad():
        for p in model.parameters():
            p -= lr * p.grad
            p.grad.zero_()
    total += loss.item()
print("SGD one-epoch | mean train loss:", total/X_train.shape[0])

SGD one-epoch | mean train loss: 0.6491900355756942


In [17]:
##학습루프
##MBGD
model = LinearReg()
lr = 1e-2
B = 64

total = 0.0
for bx, by in make_minibatches(X_train, y_train, B):
    pred = model(bx)
    loss = mse_loss(pred, by)
    loss.backward()
    with torch.no_grad():
        for p in model.parameters():
            p -= lr * p.grad
            p.grad.zero_()
    total += loss.item() * bx.shape[0]
print("MBGD one-epoch | mean train loss:", total/X_train.shape[0])

MBGD one-epoch | mean train loss: 6.649388484954834


In [12]:
##Result
plt.plot(loss_history, label="BGD")
plt.plot(loss_history_sgd, label="SGD")
plt.plot(loss_history_mbgd, label="MBGD")
plt.legend()
plt.show

NameError: name 'loss_history' is not defined